In [5]:
import sys, os, distutils.core

# Define the directory name
directory_name = 'detectron2'

# Check if the directory already exists
if not os.path.exists(directory_name):
    # Directory doesn't exist, so clone the repository
    !git clone 'https://github.com/facebookresearch/detectron2'
    
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])} -q
sys.path.insert(0, os.path.abspath('./detectron2'))

In [17]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [7]:
# Define the directory name
directory_name = 'detr'

# Check if the directory already exists
if not os.path.exists(directory_name):
    # Directory doesn't exist, so clone the repository
    !git clone https://github.com/facebookresearch/detr

%cd detr/

!python d2/converter.py --source_model https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth --output_model converted_model.pth

/sfs/qumulo/qhome/ybt7qf/ds6050-group1-project/detr
transformer.encoder.layers.0.self_attn.in_proj_weight -> detr.transformer.encoder.layers.0.self_attn.in_proj_weight
transformer.encoder.layers.0.self_attn.in_proj_bias -> detr.transformer.encoder.layers.0.self_attn.in_proj_bias
transformer.encoder.layers.0.self_attn.out_proj.weight -> detr.transformer.encoder.layers.0.self_attn.out_proj.weight
transformer.encoder.layers.0.self_attn.out_proj.bias -> detr.transformer.encoder.layers.0.self_attn.out_proj.bias
transformer.encoder.layers.0.linear1.weight -> detr.transformer.encoder.layers.0.linear1.weight
transformer.encoder.layers.0.linear1.bias -> detr.transformer.encoder.layers.0.linear1.bias
transformer.encoder.layers.0.linear2.weight -> detr.transformer.encoder.layers.0.linear2.weight
transformer.encoder.layers.0.linear2.bias -> detr.transformer.encoder.layers.0.linear2.bias
transformer.encoder.layers.0.norm1.weight -> detr.transformer.encoder.layers.0.norm1.weight
transformer.encoder.

In [8]:
from d2.detr import add_detr_config

cfg = get_cfg()

add_detr_config(cfg)
cfg.merge_from_file("d2/configs/detr_256_6_6_torchvision.yaml")

cfg.MODEL.WEIGHTS = "converted_model.pth"

In [9]:
def filter_predictions_from_outputs(outputs,
                                    threshold=0.7,
                                    verbose=True):

    predictions = outputs["instances"].to("cpu")

    if verbose:
        print(list(predictions.get_fields()))

    # Reference: https://github.com/facebookresearch/detectron2/blob/7f06f5383421b847d299b8edf480a71e2af66e63/detectron2/structures/instances.py#L27
    #
    #   Indexing: ``instances[indices]`` will apply the indexing on all the fields
    #   and returns a new :class:`Instances`.
    #   Typically, ``indices`` is a integer vector of indices,
    #   or a binary mask of length ``num_instances``

    indices = [i
            for (i, s) in enumerate(predictions.scores)
            if s >= threshold
            ]

    filtered_predictions = predictions[indices]

    return filtered_predictions

In [10]:
url = 'http://images.cocodataset.org/train2017/000000310645.jpg'
!wget $url -q -O input.jpg

im = cv2.imread('input.jpg')

In [11]:
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

[11/15 20:34:17 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from converted_model.pth ...


Some model parameters or buffers are not found in the checkpoint:
criterion.empty_weight


In [12]:
threshold = 0.7

filtered_predictions = filter_predictions_from_outputs(outputs,
                                                  threshold=threshold)

['pred_boxes', 'scores', 'pred_classes']


In [21]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(filtered_predictions)
cv2_imshow(out.get_image()[:, :, ::-1])

error: OpenCV(4.8.1) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
